# Polinómios NTRU

In [6]:
n = next_prime(2000)
q = 257

2003


In [12]:
Z = PolynomialRing(ZZ,'w')
R = PolynomialRing(GF(q),'w')
z = Z.gen()
print(z)
r = R.gen()
f = z^n - 1
fq = r^n - 1

Zq = QuotientRing(Z,f)
Rq = QuotientRing(R,fq)

w


In [45]:
t = Z(f/(z-1))

In [4]:
t.is_irreducible()

True

In [9]:
a = Rq.random_element()
print(a)

15*wbar^2002 + 10*wbar^2001 + 121*wbar^2000 + 243*wbar^1999 + 140*wbar^1998 + 238*wbar^1997 + 181*wbar^1996 + 20*wbar^1995 + 59*wbar^1994 + 228*wbar^1993 + 222*wbar^1992 + 110*wbar^1991 + 25*wbar^1990 + 26*wbar^1989 + 92*wbar^1988 + 20*wbar^1987 + 174*wbar^1986 + 118*wbar^1985 + 70*wbar^1984 + 36*wbar^1983 + 142*wbar^1982 + 101*wbar^1981 + 201*wbar^1980 + 238*wbar^1979 + 43*wbar^1978 + 158*wbar^1977 + 169*wbar^1976 + 111*wbar^1975 + 65*wbar^1974 + 11*wbar^1973 + 242*wbar^1972 + 143*wbar^1971 + 124*wbar^1970 + 109*wbar^1969 + 146*wbar^1968 + 33*wbar^1967 + 172*wbar^1966 + 17*wbar^1965 + 50*wbar^1964 + 51*wbar^1963 + 75*wbar^1962 + 198*wbar^1961 + 106*wbar^1960 + 178*wbar^1959 + 129*wbar^1958 + 75*wbar^1957 + 27*wbar^1956 + 41*wbar^1955 + 129*wbar^1954 + 162*wbar^1953 + 158*wbar^1952 + 58*wbar^1951 + 82*wbar^1950 + 8*wbar^1949 + 182*wbar^1948 + 31*wbar^1947 + 128*wbar^1946 + 8*wbar^1945 + wbar^1944 + 193*wbar^1943 + 231*wbar^1942 + 135*wbar^1941 + 85*wbar^1940 + 47*wbar^1939 + 203*wbar^1

In [47]:
Z = PolynomialRing(ZZ,'w')
w = Z.gen()
factor(w^2-2)
p = w^2-2
p / (p-1)

(w^2 - 2)/(w^2 - 3)

In [50]:
p + 1*w^3

w^3 + w^2 - 2